## pydiode (red=2) - UDP.c - 208 KiB rcv sckt buf - MTU 9194 - 9166

In [1]:
import pandas as pd

tests1 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-default/default/pydiode-2-mtu9194-9166/testresults_pydiode_2_500000000.csv')
tests2 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-default/default/pydiode-2-mtu9194-9166/testresults_pydiode_2_750000000.csv')
tests3 = pd.read_csv('https://raw.githubusercontent.com/Gert-Jan1966/DataDiodesAndPacketLoss/main/Testresults/recbuff-default/default/pydiode-2-mtu9194-9166/testresults_pydiode_2_1000000000.csv')
results = pd.concat([tests1, tests2, tests3])
results.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   bitrate            3000 non-null   int64  
 1   trial              3000 non-null   int64  
 2   sender_rc          3000 non-null   int64  
 3   receiver_rc        3000 non-null   int64  
 4   Xfer_time          3000 non-null   float64
 5   Xfer_success       3000 non-null   bool   
 6   in_file_size       3000 non-null   int64  
 7   out_file_size      3000 non-null   int64  
 8   dtgrms_sent        3000 non-null   object 
 9   dtgrms_received    3000 non-null   object 
 10  dtgrms_corrupt     3000 non-null   object 
 11  tx_errors          3000 non-null   int64  
 12  tx_dropped         3000 non-null   int64  
 13  rx_errors          3000 non-null   int64  
 14  rx_dropped         3000 non-null   int64  
 15  snmp_OutDatagrams  3000 non-null   int64  
 16  snmp_SndbufErrors  3000 non-nu

#### Transfer success vs Transfer failure

In [2]:
results['Xfer_success'].value_counts()

Xfer_success
True    3000
Name: count, dtype: int64

#### Did we lose Frames at the Link layer?
In that case, at least one of 'tx_errors', 'tx_dropped', 'rx_errors' or 'rx_dropped' should not be zero.....

In [3]:
results.query('(tx_errors != 0) | (tx_dropped != 0) | (rx_errors != 0) | (rx_dropped != 0)').value_counts()

Series([], Name: count, dtype: int64)

#### How many trials with receiver socket buffer errors? (lost Segments)

In [4]:
results.query('snmp_RcvbufErrors != 0').value_counts()

bitrate     trial  sender_rc  receiver_rc  Xfer_time  Xfer_success  in_file_size  out_file_size  dtgrms_sent  dtgrms_received  dtgrms_corrupt  tx_errors  tx_dropped  rx_errors  rx_dropped  snmp_OutDatagrams  snmp_SndbufErrors  snmp_InDatagrams  snmp_InErrors  snmp_RcvbufErrors
1000000000  999    0          0            2.207102   True          125000000     125000000      na           na               na              0          0           0          0           27294              1659               27287             4              4                    1
500000000   562    0          0            4.308981   True          125000000     125000000      na           na               na              0          0           0          0           27294              0                  27289             2              2                    1
            848    0          0            4.225844   True          125000000     125000000      na           na               na              0          0 

#### Does the number of receiver socket InErrors equal the number of receiver buffer errors?

In [5]:
results.query('snmp_InErrors != snmp_RcvbufErrors').value_counts()

Series([], Name: count, dtype: int64)

#### Do all unsuccessful trials match all trials with receiver buffer errors?

In [6]:
results.query('(snmp_RcvbufErrors != 0) & (Xfer_success == False)').value_counts()

Series([], Name: count, dtype: int64)

#### And what about snmp_SndbufErrors?

In [7]:
results.query('(snmp_SndbufErrors != 0)').value_counts()
#results.query('(snmp_SndbufErrors != 0) & (Xfer_success == True)').value_counts()

bitrate     trial  sender_rc  receiver_rc  Xfer_time  Xfer_success  in_file_size  out_file_size  dtgrms_sent  dtgrms_received  dtgrms_corrupt  tx_errors  tx_dropped  rx_errors  rx_dropped  snmp_OutDatagrams  snmp_SndbufErrors  snmp_InDatagrams  snmp_InErrors  snmp_RcvbufErrors
1000000000  960    0          0            2.199887   True          125000000     125000000      na           na               na              0          0           0          0           27294              1679               27290             1              1                    1
            961    0          0            2.213521   True          125000000     125000000      na           na               na              0          0           0          0           27294              1599               27291             0              0                    1
            962    0          0            2.196391   True          125000000     125000000      na           na               na              0          0 

#### Are there SndbufErrors without RcvbufErrors where the transfer failed?

In [8]:
results.query('(snmp_SndbufErrors != 0) & (snmp_RcvbufErrors == 0) & (Xfer_success == False)').value_counts()

Series([], Name: count, dtype: int64)